# Introduction to LLMs and Agents

Welcome to our workshop! In this session, we'll explore how to build AI-powered applications using **LangChain**, a popular framework for developing applications with Large Language Models (LLMs). We'll start with a simple chatbot and then enhance it with a multi-agent framework.

## Setting Up Our Environment

First, we need to set up our environment. We'll use OpenAI's models, so we need an API key. You can define your `OPENAI_API_KEY` in the `.env` file.

The code retrieve the key and sets some global configurations:
- `LLM_MODEL`: The specific model we'll use
- `LLM_TEMPERATURE`: Controls randomness in responses (0 means very deterministic)

In [9]:
import os

In [10]:
if not os.environ.get("OPENAI_API_KEY"):
    raise ValueError("Please set OPENAI_API_KEY environment variable")

LLM_MODEL = "gpt-4o-mini"
LLM_TEMPERATURE = 0.9

## Building a Simple ChatBot

Let's start with creating a basic chatbot using **LangChain**. We'll use:
- `ChatOpenAI`: The interface to OpenAI's chat models
- `SystemMessage`: Defines the bot's behavior and role
- `HumanMessage`: Represents user input

Our chatbot will act as a Financial Analyst. We'll create it by:
1. Instantiating the model
2. Defining a system prompt that sets the bot's role
3. Sending a user query and getting a response with `.invoke()`

This demonstrates the basic pattern of LLM interactions: prompt → response.

In [11]:
from IPython.display import Markdown
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

In [12]:
# TODO: Create a ChatOpenAI instance with the LLM model and temperature
base_model = ChatOpenAI(model=LLM_MODEL, temperature=LLM_TEMPERATURE)

In [13]:
BASE_PROMPT = """
You are a Financial Analyst. Do your best to help the client with their request based on your expertise. Give a succinct and clear response.
"""

In [14]:
# Request from the client
request = "I want to invest in the technology sector. Can you please define an investment strategy?"

# Message list for the base model
messages = [
    SystemMessage(BASE_PROMPT),
    HumanMessage(request),
]

# Invoke the model with the messages
response = base_model.invoke(messages)

In [15]:
Markdown(response.content)

Absolutely! Here’s a concise investment strategy for the technology sector:

1. **Define Your Investment Goals**: Determine your risk tolerance, time horizon, and investment objectives (e.g., growth vs. value).

2. **Diversification**: Invest across various sub-sectors within technology (e.g., software, hardware, semiconductors, cybersecurity, AI, cloud computing) to spread risk.

3. **Research and Analysis**:
   - **Fundamentals**: Look for companies with solid financials, strong revenue growth, and healthy profit margins.
   - **Trends**: Stay informed on emerging trends (like AI, 5G, and IoT) that could drive future growth.

4. **Valuation Metrics**: Use metrics such as P/E ratio, PEG ratio, and EV/EBITDA to evaluate whether stocks are overvalued or undervalued.

5. **ETFs and Mutual Funds**: Consider technology-focused ETFs or mutual funds for instant diversification and managed risk.

6. **Long-Term Focus**: Technology can be volatile; maintain a long-term perspective and avoid knee-jerk reactions to short-term market events.

7. **Regular Review**: Continuously monitor your investments and the sector for changes in fundamentals or market dynamics, adjusting your portfolio as needed.

8. **Stay Informed**: Follow industry news, earnings reports, and analyst recommendations to make informed decisions.

By following this strategy, you can position yourself well within the technology sector while managing risk effectively.

## Agentic system

We will now create a simple agentic system consisting of three agents using LangChain.
This system will help us perform a more elaborate financial analysis by including:

- Client Interface Agent: Rephrases the user’s prompt to improve the quality of the financial analyst’s response.

- Financial Analyst: Similar to the first part; provides financial advice based on the refined prompt.

- Risk Advisor: Assesses the risk associated with the advice given by the financial analyst.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Load LLM
llm = ChatOpenAI(model=LLM_MODEL, temperature=LLM_TEMPERATURE)

# Agent Prompts
client_prompt = PromptTemplate.from_template(
    "You are a Client Interface agent. A user has asked the following question:\n{user_query}\n"
    "Rephrase it clearly for a financial analyst."
)
analyst_prompt = PromptTemplate.from_template(
    "You are a Financial Analyst. The client asked: {client_rephrased_query}\n"
    "Give clear investment advice. Do not assess risk."
)
risk_prompt = PromptTemplate.from_template(
    "You are a Risk Advisor. The analyst has recommended:\n{analyst_advice}\n"
    "Evaluate this advice from a risk perspective. Offer any cautions and ways to reduce risk."
)

# Define Chains
client_chain = client_prompt | llm | StrOutputParser()
analyst_chain = analyst_prompt | llm | StrOutputParser()
risk_chain = risk_prompt | llm | StrOutputParser()

# Simulate user question
user_query = "I'm 35 and have $50,000 saved. Should I invest in real estate or index funds?"

# Step 1: Client Interface processes input
client_rephrased_query = client_chain.invoke(user_query)
print("\n🤖 Client Interface:\n", client_rephrased_query)

# Step 2: Financial Analyst responds
analyst_advice = analyst_chain.invoke(client_rephrased_query)
print("\n📈 Financial Analyst:\n", analyst_advice)

# Step 3: Risk Advisor evaluates
risk_evaluation = risk_chain.invoke(analyst_advice)
print("\n⚠️ Risk Advisor:\n", risk_evaluation)


🤖 Client Interface:
 A 35-year-old individual with $50,000 in savings is seeking advice on whether to invest in real estate or index funds. What would be the best recommendation for their investment strategy?

📈 Financial Analyst:
 When advising on whether to invest in real estate or index funds, it’s essential to consider the individual’s financial goals, timeline, and preferences.

### Recommendation:

1. **Diversification**: A balanced approach is often beneficial. Instead of committing all funds to one investment type, consider splitting the investment between real estate and index funds. For instance, allocate 60% to index funds and 40% to real estate.

2. **Index Funds**: 
   - **Liquidity**: Index funds offer high liquidity, allowing the individual to easily buy and sell shares. This is advantageous for those who may need access to their funds in the short term.
   - **Market Growth**: Index funds typically provide exposure to a diversified range of companies, which can lead to

In [18]:
summarize_prompt = PromptTemplate.from_template(
    "You are a Client interface agent. Give a clear financial strategy with precise numbers and allocations"
    "based on \n{analyst_advice}\n and \n{risk_evaluation}\n"
)

summarized_chain = {"client_rephrased_query": client_chain} | {"analyst_advice": analyst_chain} | {"risk_evaluation": risk_chain} | summarize_prompt | llm | StrOutputParser()

In [20]:
summary= summarized_chain.invoke(user_query)
print("\n⚠️ Risk Advisor:\n", summary)


⚠️ Risk Advisor:
 When deciding between investing in real estate and index funds with a budget of $50,000, a comprehensive financial strategy can be formed by analyzing the benefits and risks of each option. Below is a clear allocation recommendation based on your goals, risk tolerance, and investment horizon.

### Financial Strategy Overview

#### Total Investment Amount: $50,000

**Investment Allocation:**
1. **Real Estate Investment – $20,000 (40%)**
   - **Use**: Down payment for rental property or investment in a Real Estate Investment Trust (REIT).
   - **Assumption**: Target a property that is worth $100,000 (20% down payment).
   - **Financing**: Use a mortgage for the remaining $80,000, assuming a 30-year fixed loan with a 4% interest rate.
   - **Potential Rental Income**: Aim for a rental yield of 6%-8% annually on the property value.
   - **Ongoing Costs**: Set aside 20% of rental income for maintenance, taxes, and management fees.

2. **Index Funds Investment – $30,000 (6